# 0) Импорт всех нужных библиотек

In [2]:
!rm -rf datas

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D , UpSampling2D ,Conv2DTranspose
from imutils.object_detection import non_max_suppression
import matplotlib.pyplot as plt

device = tf.config.list_physical_devices('GPU')



---



---



# 1) Подготовка датасета

In [2]:
! rm datas150k.rar
! wget https://www.dropbox.com/s/542b4om2x4jw4dr/datas150k.rar

rm: cannot remove 'datas150k.rar': No such file or directory
--2021-04-20 19:53:50--  https://www.dropbox.com/s/542b4om2x4jw4dr/datas150k.rar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:601a:18::a27d:712
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/542b4om2x4jw4dr/datas150k.rar [following]
--2021-04-20 19:53:50--  https://www.dropbox.com/s/raw/542b4om2x4jw4dr/datas150k.rar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8e992f2f1440fc56e2753ee2e7.dl.dropboxusercontent.com/cd/0/inline/BM_3KGquXtLOm_FlirDTfrIfcf-2kxLQDi-zJisCL9mFqGHZEvEf2mShf5FiuHjawvvzrV9tO570pMFz5XrkNkyiEFsnbnoz3HH8ThFgSXdebxQtzrwzak24XnMNCih2lU8P9E9Tfdk5PiEeM5MTC6_Z/file# [following]
--2021-04-20 19:53:51--  https://uc8e992f2f1440fc56e2753ee2e7.dl.dropboxusercontent.com/cd/0/inline/BM_3KGquXtLOm_FlirDTfrIfcf

In [3]:
get_ipython().system_raw("unrar x datas150k.rar")

In [4]:
DATA_PATH = 'datas/'
IMG_SIZE = [64, 128, 3]
batch_size = 128

In [5]:
import os
classes = [f for f in os.listdir(DATA_PATH)]
classes.sort()
NUM_OF_CLASSES = len(classes)
classes

['antqua',
 'arial',
 'bahnschrift',
 'book antiqua',
 'bookman old style',
 'calibri',
 'cambria',
 'candara',
 'century',
 'century schoolbook',
 'comic sans',
 'consolas',
 'constantia',
 'corbel',
 'courier new',
 'gabriola',
 'garamond',
 'georgia',
 'gothic',
 'haettenschweiler',
 'impact',
 'lucida console',
 'lucida sans unicode',
 'microsoft sans serif',
 'mistral',
 'monotype corsiva',
 'ms rederence sans serif',
 'palatino linotype',
 'segoe print',
 'segoe script',
 'segoe ui',
 'sitka',
 'sylfaen',
 'tahoma',
 'times new roman',
 'trebuchet ms',
 'verdana']

In [6]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255.0, label

In [7]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=classes,
    batch_size=batch_size,
    image_size=tuple(IMG_SIZE[:-1]),
    validation_split=0.1,
    seed=1488,
    subset='training'
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=classes,
    batch_size=batch_size,
    image_size=tuple(IMG_SIZE[:-1]),
    validation_split=0.1,
    seed=1488,
    subset='validation'
)

Found 150000 files belonging to 37 classes.
Using 135000 files for training.
Found 150000 files belonging to 37 classes.
Using 15000 files for validation.


In [8]:
ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_validation = ds_validation.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)



---



---



# 2) Создание и обучение модели для классификации шрифтов

#### Также здесь я дополнительно обучил модель ResNet50, чтобы сравнить с ней точность работы моей модели

#### Моя модель:

##### Создание

In [9]:
def get_model(input_shape_, num_classes):
  model = keras.Sequential(
      [
       layers.experimental.preprocessing.Resizing(64, 128),

       layers.Conv2D(8, 3, padding='same', input_shape=input_shape_),
       layers.BatchNormalization(),
       layers.MaxPool2D(strides=(2,2)),
       layers.ReLU(),

       layers.Conv2D(16, 3, padding='same'),
       layers.BatchNormalization(),
       layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
       layers.ReLU(),

       layers.Conv2D(32, 3, padding='same'),
       layers.BatchNormalization(),
       layers.MaxPool2D(strides=(2,2)),
       layers.ReLU(),

       layers.Conv2D(16, 3, padding='same'),
       layers.BatchNormalization(),
       layers.MaxPool2D(strides=(2,2)),
       layers.ReLU(),

      #  layers.Conv2D(64, 3, padding='same'),
      #  layers.BatchNormalization(),
      #  layers.MaxPool2D(strides=(2,2)),
      #  layers.ReLU(),
       
      #  layers.Conv2D(128, 3, padding='same', input_shape=[16, 32, 64], data_format='channels_last'),
      #  layers.BatchNormalization(),
      #  layers.MaxPool2D(strides=(2,2)),
      #  layers.ReLU(),

      #  layers.Conv2D(128, 3, padding='same', input_shape=[8, 16, 128], data_format='channels_last'),
      #  layers.BatchNormalization(),
      #  layers.MaxPool2D(strides=(2,2)),
      #  layers.ReLU(),

       layers.Flatten(),
       layers.Dense(256, activation='relu',
                    ),
       layers.Dense(256, activation='relu',
                    ),
       layers.Dropout(0.4),
       layers.Dense(32, activation='relu',
                    ),
       layers.Dense(40, activation='relu',
                    ),
       layers.Dropout(0.15),
       layers.Dense(num_classes, activation='softmax', 
                  # kernel_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.01)
                  )
      ]
  )
  model.build([None] + input_shape_)
  return model

In [10]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-3,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=False)
model = get_model(IMG_SIZE, NUM_OF_CLASSES)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.TopKCategoricalAccuracy(k=3)],)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 64, 128, 3)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 128, 8)        224       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 128, 8)        32        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 64, 8)         0         
_________________________________________________________________
re_lu (ReLU)                 (None, 32, 64, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 64, 16)        1168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 64, 16)        6

##### Обучение и тест

In [16]:
model.fit(ds_train, batch_size=batch_size, epochs=8, verbose=1)

Epoch 1/8
1055/1055 [==============================] - 118s 110ms/step - loss: 2.1835 - top_k_categorical_accuracy: 0.5714
Epoch 2/8
1055/1055 [==============================] - 123s 116ms/step - loss: 1.1689 - top_k_categorical_accuracy: 0.8517
Epoch 3/8
1055/1055 [==============================] - 124s 117ms/step - loss: 0.9244 - top_k_categorical_accuracy: 0.8971
Epoch 4/8
1055/1055 [==============================] - 124s 117ms/step - loss: 0.7864 - top_k_categorical_accuracy: 0.9212
Epoch 5/8
1055/1055 [==============================] - 126s 118ms/step - loss: 0.7053 - top_k_categorical_accuracy: 0.9335
Epoch 6/8
1055/1055 [==============================] - 132s 125ms/step - loss: 0.6473 - top_k_categorical_accuracy: 0.9416
Epoch 7/8
1055/1055 [==============================] - 131s 123ms/step - loss: 0.5974 - top_k_categorical_accuracy: 0.9481
Epoch 8/8
1055/1055 [==============================] - 130s 123ms/step - loss: 0.5607 - top_k_categorical_accuracy: 0.9532


In [17]:
loss, acc = model.evaluate(ds_validation)
acc

118/118 [==============================] - 14s 110ms/step - loss: 0.5027 - top_k_categorical_accuracy: 0.9564


0.9563999772071838



---



---



#### ResNet50V2:

In [ ]:
m = keras.applications.ResNet50V2(weights=None, input_shape=IMG_SIZE, classes=NUM_OF_CLASSES)
m.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.TopKCategoricalAccuracy(k=3)],)
m.summary()

In [ ]:
m.fit(ds_train, batch_size=batch_size, epochs=8, verbose=1)

Epoch 1/2
1055/1055 [==============================] - 255s 241ms/step - loss: 0.1906 - top_k_categorical_accuracy: 0.9918
Epoch 2/2
1055/1055 [==============================] - 259s 245ms/step - loss: 0.1660 - top_k_categorical_accuracy: 0.9938


In [ ]:
loss, acc = m.evaluate(ds_validation)
acc

118/118 [==============================] - 11s 90ms/step - loss: 0.4359 - top_k_categorical_accuracy: 0.9796


0.9796000123023987

Из сравнения видно, что наша модель несильно уступает модели ResNet50V2 в качестве, но при этом она значительно меньше по объему(ResNet50V2 - 90мб, наша модель - 800кб)

---



---



# 3) Код для постобработки выхода модели для детекции
### Будет использоваться в проекте, но не в этом ноутбуке

Код в следующей ячейке взят из [этого ноутбука](https://colab.research.google.com/github/sayakpaul/Adventures-in-TensorFlow-Lite/blob/master/EAST_TFLite.ipynb)

In [11]:
def post_process(score, geo, ratioW, ratioH):
    (numRows, numCols) = score.shape[2:4]
    rects = []
    confidences = []

    for y in range(0, numRows):
        scoresData = score[0, 0, y]
        xData0 = geo[0, 0, y]
        xData1 = geo[0, 1, y]
        xData2 = geo[0, 2, y]
        xData3 = geo[0, 3, y]
        anglesData = geo[0, 4, y]

        for x in range(0, numCols):

            (offsetX, offsetY) = (x * 4.0, y * 4.0)

            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)

            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]

            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)

            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])

    selected_indices = tf.image.non_max_suppression(rects, confidences, max_output_size=5, iou_threshold=0.3, score_threshold=0.5)
    boxes = np.array(tf.gather(rects, selected_indices))

    boxes2 = []
    for (startX, startY, endX, endY) in boxes:
        startX = min(max(int(startX * ratioW), 0), W)
        startY = min(max(int(startY * ratioH), 0), H)
        endX = min(max(int(endX * ratioW), 0), W)
        endY = min(max(int(endY * ratioH), 0), H)
        boxes2.append((startX, startY, endX, endY))
    return np.array(boxes2)



---



---



# 4) Сохранение модели в формате .tflite


In [13]:
model_index = 0

In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model{}({}%).tflite'.format(model_index, int(acc*100)), 'wb') as f:
  f.write(tflite_model)
  model_index += 1

INFO:tensorflow:Assets written to: /tmp/tmp_ryif3_9/assets
